In [1]:
import os
from dotenv import load_dotenv

import requests
import pandas as pd
from datetime import date, timedelta
import json

## 1. exchangerate.host --> FX-re jo lehet, ingyen tier lefedi teljesen ami kell (Free 100 monthly request)
### Timeframe endpoint
Igazabol idöintervallumot lehet megadni, ez teljesen jo lesz egy alkalmankenti frissiteshez.
#### Fontos:
- van egy `start_date` es egy `end_date` key benne
- van `timeframe` --> boolean mezöje
- a `quotes`-on belul nem egyböl a currency parok jönnek, hanem a historikus datumok az alkey-ek es csak azon belul vannak a FX parok --> quotes --> date --> FXpairs
- max 365 napot fedhet le a timeframe
- `1999-01-01`-töl vannak adatok

In [2]:
# Betölti a projekt gyökerében lévő .env-t
load_dotenv()
access_key = os.getenv("API_KEY_FX")
print("API KEY loaded:", access_key is not None)

# több endpoint van görgess lejebb a historikusert
endpoint = 'timeframe'
# jelenlegi ev es abbol szarmaztatott start es end date-ek
date = date.today() - timedelta(days=1)
start_date = f"{date.year}-01-01"
end_date = f"{date}"

## 100 free request per month
response = requests.get('https://api.exchangerate.host/' +
                        endpoint +
                        '?access_key=' + access_key +
                        '&start_date=' + start_date + 
                        '&end_date=' + str(end_date)
                        )

data_tf = response.json()

API KEY loaded: True


In [3]:
with open(f"../jsons/fx_jsons/usd_other_fx_rates_{date.year}.json", "w", encoding="utf-8") as f:
    json.dump(data_tf, f, indent=4, ensure_ascii=False)

In [4]:
# beolvasom a JSON file tartalmat, es be is zarja attol fuggetlenul, h sikeres volt e a megnyitas.
with open(f"../jsons/fx_jsons/usd_other_fx_rates_{date.year}.json", "r") as f:
    data = json.load(f)

# csak a quotes kinyerese
quotes = data["quotes"]
df = pd.DataFrame.from_dict(quotes, orient="index")

# Quick QC
print("Year starting date: ", df.index[0])
print("Year ending date: ", df.index[-1])

delta_day = 0
for i in range(len(df.index)):
    if i > 0:
        diff = (pd.to_datetime(df.index[i])-pd.to_datetime(df.index[i-1])).days
        if diff > delta_day:
            delta_day = diff
print("Maximum difference between dates in dataframe:", delta_day)

df.tail()

Year starting date:  2025-01-01
Year ending date:  2025-11-30
Maximum difference between dates in dataframe: 1


,USDAED,USDAFN,USDALL,USDAMD,USDANG,USDAOA,USDARS,USDAUD,USDAWG,USDAZN,...,USDXDR,USDXOF,USDXPF,USDYER,USDZAR,USDZMK,USDZMW,USDZWL,USDSTN,USDXCG
2025-11-26,3.672503,65.503851,83.350020,381.119940,1.790055,916.999703,1452.656198,1.533050,1.8025,1.706540,...,0.704694,564.999653,103.296933,238.350326,17.146105,9001.201748,22.907755,321.999592,21.380000,1.802926
2025-11-27,3.672499,66.296435,83.452716,381.069676,1.790055,916.999889,1450.143795,1.530093,1.8025,1.697048,...,0.702676,566.097512,102.922584,238.349720,17.150995,9001.194061,22.997493,321.999592,21.143656,1.802044
2025-11-28,3.672504,66.111997,83.269388,379.445618,1.790055,916.000367,1450.250402,1.534213,1.8000,1.703970,...,0.703367,565.553304,102.823641,238.303589,17.111925,9001.203584,22.859853,321.999592,21.123421,1.797129
2025-11-29,3.672504,66.111997,83.269388,379.445618,1.790055,916.000367,1450.250402,1.534213,1.8000,1.703970,...,0.703367,565.553304,102.823641,238.303589,17.111925,9001.203584,22.859853,321.999592,21.123421,1.797129
2025-11-30,3.672502,66.111997,83.269388,379.445618,1.790055,915.999953,1450.249746,1.526473,1.8000,1.700451,...,0.703367,565.553304,102.823641,238.298035,17.121665,9001.198714,22.859853,321.999592,21.123421,1.797129
